# CARTE-Enbridge Bootcamp
#### Lab 2-2

Transforming customer service using AI

In this lab, we will use HuggingFace's transformers library to build a customer service chatbot. We will use the DistilBERT model to build a chatbot that can answer questions about the customer service policy of a company.

For this notebook to run in a reasonable time, it's essential that you enable GPU acceleration. To do this, go to Runtime > Change runtime type, and select GPU as the hardware accelerator.

In [ ]:
# Check if GPU is enabled
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
elif torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f'Running on a Mac with Metal Performance Shaders (MPS).')
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead (NOT RECOMMENDED!)')

First, we will import the necessary libraries and modules. If we are running this notebook on Google Colab, we will install the libraries that we need in the cell below. We are installing directly from GitHub, to ensure that we have the latest version of each library.

In [ ]:
# Check if we are running on Google Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q -U transformers datasets evaluate accelerate

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

## Load the dataset

Using the load_dataset() function from the datasets library, we will load the customer service dataset. We will be using a dataset of customer service queries that is designed for use in training chatbots.

The parts of the dataset that we are interested in are the customer's query and the 'intent' of the customer's query. The intent is the purpose of the customer's query. For example, if the customer's query is "What is your return policy?", the intent is "return policy". This makes it easier for the interaction to be routed to the correct department, or for collecting metrics on the types of queries that customers have.

In [ ]:
data = load_dataset('bitext/Bitext-customer-support-llm-chatbot-training-dataset')

Let's take a quick look at a few random samples from the dataset. We will print the customer's query, the intent, and the response from the chatbot.

In [ ]:
from random import choice
for i in range(5):
    sample = choice(data['train'])
    print(f'Question: {sample["instruction"]}')
    print(f'Customer Intent: {sample["intent"]}')
    print('---'*10)

## Preprocess the data

Now that we have our data downloaded, the next step is to preprocess it. For any model, each word in the model's vocabulary is assigned a unique number. The model doesn't see the actual words in the text, it only sees the numbers. So, we need to convert the text into numbers. This is called tokenization. Because we are fine-tuning an existing model, we can simply load the tokenizer that was created for that model. The tokenizer will convert the text into numbers, and also add any special tokens that the model needs. For example, the DistilBERT model needs a [CLS] token at the beginning of the text, and a [SEP] token at the end of the text. The tokenizer will add these tokens for us.



In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

Let's see what happens when we tokenize a sample text:

In [ ]:
question = "What is your return policy?"
tokens = tokenizer.encode(question)
print(tokens)

Each of these values corresponds to one of the words in the text. We can use the tokenizer to convert these numbers back into words, and see what the tokenizer did to the text.

In [ ]:
# Convert the tokens back into words
print(tokenizer.convert_ids_to_tokens(tokens))

As we can see, the tokenizer converted the text into numbers, and added the [CLS] and [SEP] tokens.

**Your Turn**

Try tokenizing a few other sample texts. 
- What happens if you put in a nonsense word? 
- What happens if you put in a text that's longer than the maximum length of the model (512 tokens)? 
- What happens if you put in an emoji, or another character that's not in the model's vocabulary?

In [ ]:
def encode_and_decode(question):
    tokens = tokenizer.encode(question)
    recons = tokenizer.convert_ids_to_tokens(tokens)
    for token, id in zip(recons, tokens):
        print(f'{token:<12} {id}')
    print()

In [ ]:
encode_and_decode("What is your return policy?")

We are ready to tokenize our dataset. We will use the map() function to apply the tokenizer to each sample in the dataset. We will also set the maximum length of the input to 512 tokens.

There are a couple of parameters that we need to set. The first is padding. Although our input sentences can be any length, the model requires input to be a consistent size. The tokenizer can 'pad' the input by adding special tokens to the end of the input. The model will ignore these tokens, but they will allow the input to be the same size. 

The second parameter is truncation. If the input is longer than the maximum length, the tokenizer will truncate the input. We will set both of these parameters to True.


In [ ]:
def tokenize(batch):
    return tokenizer(batch['instruction'], padding=True, truncation=True)

tokenized_dataset = data.map(tokenize, batched=True, batch_size=len(data), remove_columns=data['train'].column_names) # We don't want to keep the original text around, so we remove it.

The intents that we are trying to predict are text labels, like `payment_issue`, but the model is going to predict numbers. We can assign a number to each label, and then create a mapping from the label to the number. We will use this mapping to convert the labels into numbers.

In [ ]:
intents = set(data['train']['intent']) # Get all of the unique intents
intents = list(intents) # Convert to a list, so that the order is consistent
intents.sort() # Sort alphabetically

id2label = {i: label for i, label in enumerate(intents)} # Create a mapping from the index to the label
label2id = {label: i for i, label in enumerate(intents)} # Create a mapping from the label to the index

print(id2label)

In [ ]:
ids = [label2id[label] for label in data['train']['intent']] # Get a list of all the labels as numbers
tokenized_dataset = tokenized_dataset['train'].add_column('label', ids) # Add the labels to the dataset

We are almost finished preparing our data! The last thing we need to do is split the dataset into a training set and a test set. We will use 80% of the data for training, and 20% for testing.

In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

We can confirm that the data looks how we expect by printing out the object:

In [ ]:
tokenized_dataset

We also load a "data collator". This is a function that combines the data into batches, to make it easier for the model to process. We will use the default data collator, which simply combines the data into batches, and adds padding to the end of the input.

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Fine-tune the model

Now that we have our data ready, we can fine-tune the model. As with the earlier stages, HuggingFace makes it very easy to load an existing model. Because the task we are trying to solve is text classification, we will use the AutoModelForSequenceClassification class. This class will load a model that has already been trained on a text classification task, and fine-tune it on our dataset. We will use the DistilBERT model, which is a smaller version of the BERT model. This will allow us to train the model faster, and use less memory.

There are a few arguments that we have to pass:
- The model name. We will use the DistilBERT model, which is a smaller version of the BERT model.
- The number of labels (or possible intents) in our dataset. This is the number of outputs that the model will have. We can get this number from the length of the intents list.
- The mapping from the label to the index. This is the mapping that we created earlier, which converts the label to a number.
- The mapping from the index to the label. This is the mapping that we created earlier, which converts the number back into a label.

You might get a warning that some of the weights are not initialized. This is fine, because we are fine-tuning the model, so these weights will be updated during training.

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=len(intents), id2label=id2label, label2id=label2id)

Next, we define the arguments for training the model. Each argument is explained in the code below.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,                # How often to log results
    evaluation_strategy='epoch',     # How often to evaluate the model - once per epoch
    save_strategy='epoch',           # How often to save the model - once per epoch
    load_best_model_at_end=True,     # At the end of training, load the model from the epoch with the best validation loss
)

# Move everything to the GPU/MPS, if available
model.to(device)

trainer = Trainer(
    model=model,                                 # the model to be trained
    args=training_args,                          # training arguments, defined above
    train_dataset=tokenized_dataset['train'],    # training dataset
    eval_dataset=tokenized_dataset['test'],      # evaluation dataset
    data_collator=data_collator,                 # data collator
)

Now we can train the model! This will take around 5 minutes on Google Colab.

In [ ]:
trainer.train()

In [ ]:
# You can save your model to disk
trainer.save_model("./models")

In [ ]:
# ...and reload it at any time in the future!
model = AutoModelForSequenceClassification.from_pretrained("./models")

In [ ]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model, tokenizer=tokenizer)

In [ ]:
# Let's try it out!
classifier("What is your return policy?")

**Your Turn**

Try asking a few questions to the chatbot. 

- What happens if you ask a question that is not in the dataset? 
- What happens if you ask a question that is in the dataset, but is phrased differently?
- What happens if you ask a question that has a typo? Or a word that is not in the model's vocabulary?
- What happens if you ask a question that is not in English?

In [ ]:
classifier("What is your return policy?")

## Model Evaluation

Now that we have trained our model, we can evaluate it. We will get the predictions for the test set, and calculate the accuracy.

In [ ]:
from sklearn.metrics import accuracy_score

predictions = trainer.predict(tokenized_dataset['test'])
predictions = predictions.predictions.argmax(-1)
labels = tokenized_dataset['test']['label']

accuracy = accuracy_score(labels, predictions)
print(f'Accuracy: {accuracy:.2f}')

Wow! As we can see, the model is very accurate. This is because we are using a model that has already been trained on a similar task. If we were training a model from scratch, we would expect the accuracy to be much lower.

**Your Turn**

When evaluation performance of a new model, it's often helpful to look at the examples that the model gets wrong. This can help you understand what types of errors the model is making, and how you might improve the model.

In the cell below, print out the examples that the model gets wrong. You can use the `predictions` and `labels` variables from the previous cell.

- What types of errors is the model making?
- Are there any patterns in the errors that the model is making?

**Bonus Task**

Try fine-tuning a different model, such as [BERT](https://huggingface.co/transformers/model_doc/bert.html), [RoBERTa](https://huggingface.co/transformers/model_doc/roberta.html), or [XLNet](https://huggingface.co/transformers/model_doc/xlnet.html). How does the accuracy change? How does the training time change? __Hint: You can change the model name in the cell where we load the model.__